In [14]:
import pandas as pd
import yfinance as yf
from backtesting import Backtest, Strategy
from backtesting.test import SMA
from backtesting.lib import crossover


stock = yf.Ticker("BP.L")
data = stock.history(period="10y")


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [16]:
bt = Backtest(data, SmaCross, commission=0.004)
stats = bt.run()
stats

Start                     2014-02-17 00:00...
End                       2024-02-16 00:00...
Duration                   3651 days 00:00:00
Exposure Time [%]                   98.611111
Equity Final [$]                  2866.501707
Equity Peak [$]                  11033.407309
Return [%]                         -71.334983
Buy & Hold Return [%]               74.799701
Return (Ann.) [%]                  -11.745833
Volatility (Ann.) [%]               24.357223
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -75.370004
Avg. Drawdown [%]                  -17.418364
Max. Drawdown Duration     3068 days 00:00:00
Avg. Drawdown Duration      515 days 00:00:00
# Trades                                  139
Win Rate [%]                        36.690647
Best Trade [%]                      26.477985
Worst Trade [%]                    -15.889408
Avg. Trade [%]                    

In [17]:
bt.plot()

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='3035', ...)

In [18]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

/Users/ivankyulev/Projects/Notebooks/.venv/lib/python3.11/site-packages/backtesting/backtesting.py:1375: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


CPU times: user 2.93 s, sys: 13.4 ms, total: 2.94 s
Wall time: 2.95 s


Start                     2014-02-17 00:00...
End                       2024-02-16 00:00...
Duration                   3651 days 00:00:00
Exposure Time [%]                    99.52381
Equity Final [$]                 10494.182168
Equity Peak [$]                  13617.060803
Return [%]                           4.941822
Buy & Hold Return [%]               74.799701
Return (Ann.) [%]                    0.483525
Volatility (Ann.) [%]               28.842195
Sharpe Ratio                         0.016764
Sortino Ratio                        0.025457
Calmar Ratio                         0.008499
Max. Drawdown [%]                  -56.892686
Avg. Drawdown [%]                   -8.313594
Max. Drawdown Duration     2577 days 00:00:00
Avg. Drawdown Duration      191 days 00:00:00
# Trades                                  266
Win Rate [%]                        37.969925
Best Trade [%]                      29.156429
Worst Trade [%]                     -13.07986
Avg. Trade [%]                    